In [ ]:
## ---------------- SEP5 testing gedi extraction code -----------------------

In [ ]:
##
# conda install -c conda-forge r-optmatch
# conda install -c conda-forge r-RItools
##

In [1]:
install.packages("s3")
install.packages("doParallel")
install.packages("RItools")    

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependency ‘svd’


Warning message in install.packages("RItools"):
“installation of package ‘svd’ had non-zero exit status”
Warning message in install.packages("RItools"):
“installation of package ‘RItools’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library("terra")
library("dplyr")
library("sf")
#install.packages("s3")
library("s3")

library("sp")
library("foreach")
library("stringr")
library("aws.s3")
#conda install -c conda-forge r-optmatch #r-ggmap r-hrbrthemes r-Hmisc
#library("optmatch")
#install.packages("doParallel")
library("doParallel")
#install.packages("RItools")    
#library("RItools")


terra 1.7.29


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.11.2, GDAL 3.7.0, PROJ 9.2.0; sf_use_s2() is TRUE

Loading required package: iterators

Loading required package: parallel



In [3]:
#s3 <- paws::s3()

In [4]:
#To test, we define the variables manually. For final version, run the commented out section below
#iso3 <-"Bpt"
iso3 <-"Bpt"
gediwk <- 24
mproc <- 1

In [5]:
#-------------------------------------------------------------------------------
#args = commandArgs(trailingOnly=TRUE)
#if (length(args)==0) {
#  stop("At least one argument must be supplied (input file).n", call.=FALSE)
#} else if (length(args)>=1) {
#  
#  iso3 <- args[1]  #country to process
#  gediwk <- args[2]   #the # of weeks GEDI data to use
#  mproc <- as.integer(args[3])  #the number of cores to use for matching 
#}
#-------------------------------------------------------------------------------

In [6]:
cat("Step 0: Loading global variables for", iso3,"with wk", gediwk, "data \n")

#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

matching_tifs <- c("wwf_biomes","wwf_ecoreg","lc2000","d2roads", "dcities","dem",
                   "pop_cnt_2000","pop_den_2000","slope", "tt2cities_2000", "wc_prec_1990-1999",
                   "wc_tmax_1990-1999","wc_tavg_1990-1999","wc_tmin_1990-1999" )

ecoreg_key <- read.csv(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))
#unlink(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))

allPAs <- readRDS(s3_get(paste(f.path,"WDPA_shapefiles/WDPA_polygons/",iso3,"_PA_poly.rds",sep="")))

MCD12Q1 <- rast(s3_get(paste(f.path,"GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif",sep="")))
crs(MCD12Q1)  <- "epsg:6933"

world_region <- rast(s3_get(paste(f.path,"GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif",sep="")))
crs(world_region)  <- "epsg:6933"

s3_get_files(c(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".shx",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".prj",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".dbf",sep="")),confirm = FALSE)
adm <- st_read(s3_get(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep="")))

#s3_path <- paste("/vsis3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/",iso3,".shp",sep="") #Redo this for the gpkg
#adm <- st_read(s3_path)
adm_prj <- project(vect(adm), "epsg:6933")

load(s3_get(paste(f.path,"rf_noclimate.RData",sep="")))
#source(s3_get(paste(f.path,"matching_func.R",sep="")))
source(s3_get(paste(f.path,"vl_GEDI-PA_2024/matching_func_2024.R",sep="")))

flag <- "run all"
#flag <- "run remaining"

Step 0: Loading global variables for Bpt with wk 24 data 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_shapefiles/WDPA_polygons/Bpt_PA_poly.rds already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_shapefiles/WDPA_polygons/Bpt_PA_poly.rds

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINE

Reading layer `Bpt' from data source 
  `/projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/Bpt.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 7 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -59.18886 ymin: -22.11834 xmax: -54.92231 ymax: -15.52386
Geodetic CRS:  WGS 84


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/vl_GEDI-PA_2024/matching_func_2024.R already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/vl_GEDI-PA_2024/matching_func_2024.R



In [7]:
#---------------STEP5. GEDI PROCESSING ---------------- 
#using GEDI shots to extract the treatment/control status, also extract the MODIS PFT for AGB prediction

# if (file.exists(paste(f.path,"WDPA_GEDI_extract/",iso3,"_wk",gediwk,"/",iso3,"_gedi_extracted_matching_wk",gediwk,".RDS", sep=""))){
cat(paste("Step 5: Performing WK ", gediwk, "GEDI extraction for", iso3,"\n"))
#matched_all <-read.csv(paste(f.path,"WDPA_extract4_residual_PAs/", iso3, "_wk_", gediwk, "_null_matches_rerun.csv",sep="")) 
matched_all <- read.csv(s3_get(paste(f.path,"WDPA_matching_results/",iso3,"_wk24_filelist.csv",sep="")))[,2]
length(matched_all)   

Step 5: Performing WK  24 GEDI extraction for Bpt 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24_filelist.csv already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24_filelist.csv



[1] 29

In [8]:
#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

matched_PAs <- foreach(this_rds=matched_all, .combine = c, .packages=c('sp','magrittr', 'dplyr','tidyr','terra')) %do% {  #non-NA matched results
  matched_PAs=c()
  print(this_rds)
  if(nchar(iso3)>3){
    id_pa <- basename(this_rds)%>%readr::parse_number() %>% unique()  
  } else {
    id_pa <- basename(this_rds)%>%readr::parse_number() %>% unique()
  }
  matched <- readRDS(s3_get(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_pa_",id_pa,"_matching_results_wk",gediwk,".RDS",sep="")))
  if(!is.null(matched)){
    if(nrow(matched)!=0){
      matched_PAs=c(matched_PAs,this_rds) 
    }
  }else {
    # print(this_rds)
    matched_PAs=matched_PAs
  }
  return(matched_PAs)
}
length(matched_PAs)


Attaching package: ‘magrittr’


The following objects are masked from ‘package:terra’:

    extract, inset



Attaching package: ‘tidyr’


The following object is masked from ‘package:magrittr’:

    extract


The following object is masked from ‘package:terra’:

    extract




[1] "Bpt_pa_220297_matching_results_wk24.RDS"


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_220297_matching_results_wk24.RDS already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_220297_matching_results_wk24.RDS



[1] "Bpt_pa_2220_matching_results_wk24.RDS"


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_2220_matching_results_wk24.RDS already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_2220_matching_results_wk24.RDS



...

[1] "Bpt_pa_555682279_matching_results_wk24.RDS"


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_555682279_matching_results_wk24.RDS already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_555682279_matching_results_wk24.RDS



[1] "Bpt_pa_67816_matching_results_wk24.RDS"


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_67816_matching_results_wk24.RDS already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_67816_matching_results_wk24.RDS



[1] 26

In [9]:
#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

flag <- "run all"
#flag <- "run remaining"

if(flag=="run all"){  #determine how many PAs to run the extraction process
  matched_PAs <- matched_PAs
  cat("Step 5: running extraction on all", length(matched_PAs),"of non-NA matched results in", iso3,"\n")
} else if (flag=="run remaining"){
  pattern1 = c(paste("wk",gediwk,sep=""),"RDS")
  extracted_PAid <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,"/",sep=""), full.names = F, pattern=paste0(pattern1, collapse="|"))%>%
    readr::parse_number() %>% unique()
  matched_PA_id <- matched_PAs %>% readr::parse_number()
  runPA_id <- matched_PA_id[!(matched_PA_id %in% extracted_PAid)]
  if (length(runPA_id)>0){
    Pattern2 <-  paste(runPA_id, collapse="|")
    runPA <-  matched_PAs[grepl(Pattern2,matched_PAs)]
    # runPA_ind <- str_detect(matched_PAs, paste(runPA_id, collapse = "|"))
    matched_PAs <-runPA
  } else {
    matched_PAs <- NULL
    cat("Step 5 already done for", iso3, "\n")
  }
}

matched_PAs

Step 5: running extraction on all 26 of non-NA matched results in Bpt 


[1] "Bpt_pa_220297_matching_results_wk24.RDS"   
 [2] "Bpt_pa_2220_matching_results_wk24.RDS"     
 [3] "Bpt_pa_2581_matching_results_wk24.RDS"     
 [4] "Bpt_pa_33968_matching_results_wk24.RDS"    
 [5] "Bpt_pa_33985_matching_results_wk24.RDS"    
 [6] "Bpt_pa_352016_matching_results_wk24.RDS"   
 [7] "Bpt_pa_352021_matching_results_wk24.RDS"   
 [8] "Bpt_pa_352270_matching_results_wk24.RDS"   
 [9] "Bpt_pa_352271_matching_results_wk24.RDS"   
[10] "Bpt_pa_352430_matching_results_wk24.RDS"   
[11] "Bpt_pa_352461_matching_results_wk24.RDS"   
[12] "Bpt_pa_352463_matching_results_wk24.RDS"   
[13] "Bpt_pa_352466_matching_results_wk24.RDS"   
[14] "Bpt_pa_478452_matching_results_wk24.RDS"   
[15] "Bpt_pa_555542500_matching_results_wk24.RDS"
[16] "Bpt_pa_555542552_matching_results_wk24.RDS"
[17] "Bpt_pa_555542777_matching_results_wk24.RDS"
[18] "Bpt_pa_555576153_matching_results_wk24.RDS"
[19] "Bpt_pa_555576191_matching_results_wk24.RDS"
[20] "Bpt_pa_555576295_matching_results_wk24.RDS"
[21] "Bpt_pa_555576310_matching_results_wk24.RDS"
[22] "Bpt_pa_555576396_matching_results_wk24.RDS"
[23] "Bpt_pa_555599959_matching_results_wk24.RDS"
[24] "Bpt_pa_555600164_matching_results_wk24.RDS"
[25] "Bpt_pa_555682279_matching_results_wk24.RDS"
[26] "Bpt_pa_67816_matching_results_wk24.RDS"

In [10]:
mproc

[1] 1

In [ ]:
registerDoParallel(cores=round(mproc))
getDoParWorkers()
startTime <- Sys.time()

foreach(this_rds=matched_PAs, .combine = foreach_rbind, .packages=c('sp','magrittr', 'dplyr','tidyr','raster')) %dopar% {
  cat("Extracting for no. ", match(this_rds,matched_PAs),"pa out of", length(matched_PAs),"\n")
#  id_pa <- this_rds %>% str_split("_") %>% unlist %>% .[3]
  id_pa <- basename(this_rds) %>% readr::parse_number() %>% unique()
#  matched <- readRDS(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_pa_", id_pa,"_matching_results_wk",gediwk,".RDS", sep=""))
  matched <- readRDS(s3_get(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_pa_",id_pa,"_matching_results_wk",gediwk,".RDS",sep="")))
  matched$pa_id <- rep(id_pa, nrow(matched))

  if (is.null(matched)==TRUE  | nrow(matched)==0) {
    cat("Matched result is null for PA", id_pa, "quitting...\n")
  } else if (!is.null(matched)==TRUE){
    mras  <- tryCatch(matched2ras(matched),
                      error=function(cond){
                        message(cond)
                        cat("Matched result is likely null for country", iso3,"pa", id_pa, "dimension of the match is", dim(matched),"\n")
                        # writeLines("Matched results is likely null for country", paste(f.path,"WDPA_log/",iso3,"_log_matching.txt", sep=""))
                        return(NULL)}) #convert the macthed df to a raster stack 
    print(table(mras$status[]))
    
    if(table(mras$status[])[2]==0 | table(mras$status[])[1]==0 | is.null(mras)){
      cat("Rasterized results unbalanced for PA", id_pa, "quitting...\n")
    } else {
      startTime <- Sys.time()
      #iso_matched_gedi<- extract_gedi(matched=matched, mras = mras)  #run filtered csvs on mras for extarction
      iso_matched_gedi <- extract_gedi(matched=matched, mras=mras, iso3=iso3)
      tElapsed <- Sys.time()-startTime
      cat(tElapsed, "for extracting all PAs in", iso3,"\n")
      # iso_matched_gedi_sub <- iso_matched_gedi %>%
      #   dplyr::select("pa_id","shot_number","status","DESIG_ENG.x","wwfbiom","wwfecoreg","PADDD","pft","region","lon_lowestmode","lat_lowestmode",
      #                 "rh_010","rh_020", "rh_030", "rh_040",  "rh_050",  "rh_060" , "rh_070","rh_075",  "rh_080",  "rh_090",  "rh_098", "AGBD","cover","pai","REGION","PFT")  #write to individual country folder
      iso_matched_gedi <-  iso_matched_gedi %>%
            dplyr::select("pa_id","status","shot_number", "lat_lowestmode", "lon_lowestmode", "wwfbiom","wwfecoreg",
                          "pft","region", "rh25", "rh50", "rh75","rh90", "rh98", "agbd", "agbd_se")
    if (length(unique(iso_matched_gedi$wwfbiom)) >1){
        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% gsub("\\b(\\p{L})\\p{L}{2,}|.","\\U\\1",.,perl = TRUE)%>% str_c( collapse = "+")
    } else if (length(unique(iso_matched_gedi$wwfbiom))==1){
        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% gsub('\\b(\\p{L})\\p{L}{2,}|.','\\U\\1',.,perl = TRUE)
    } else {
        pabiome <- iso_matched_gedi$wwfbiom %>% unique()
    }
    # papaddd <- unique(iso_matched_gedi$PADDD) %>% getmode()
    continent <- unique(iso_matched_gedi$region) %>% getmode()
    print(paste('output df',dim(iso_matched_gedi)))

    #dir.create(file.path(paste("output/WDPA_extract/",iso3,"_wk",gediwk,"/",sep="")),recursive=TRUE)
    #saveRDS(iso_matched_gedi, file=paste("output/WDPA_extract/",iso3,"_wk",gediwk,"/",iso3,"_pa_", id_pa,"_gedi_wk_",gediwk,"_conti_","biome_",pabiome,".RDS", sep=""))
    #write.csv(iso_matched_gedi, file=paste("output/WDPA_extract/",iso3,"_wk",gediwk,"/",iso3,"_pa_", id_pa,"_iso_matched_gedi_sub_wk_",gediwk,".csv", sep=""))
      dir.create(file.path(paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/",iso3,"_wk",gediwk,"/",sep="")),recursive=TRUE)
      saveRDS(iso_matched_gedi, file=paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/",iso3,"_wk",gediwk,"/",iso3,"_pa_", id_pa,"_gedi_wk_",gediwk,"_conti_","biome_",pabiome,".RDS", sep=""))
      write.csv(iso_matched_gedi, file=paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/",iso3,"_wk",gediwk,"/",iso3,"_pa_", id_pa,"_iso_matched_gedi_sub_wk_",gediwk,".csv", sep=""))
      cat(id_pa,"in",iso3,"result is written to dir\n")
    }
  }
  return(NULL)
}

stopImplicitCluster()
tElapsed <- Sys.time()-startTime
cat(tElapsed, "for extracting all PAs in", iso3,"\n")
cat("Done GEDI extraction for pa in ",iso3,"\n")    



[1] 1


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select




Extracting for no.  1 pa out of 26 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_220297_matching_results_wk24.RDS already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24/Bpt_pa_220297_matching_results_wk24.RDS



Bpt converting the matched csv to a raster stack for extraction

   0    1 
1126 1126 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_Bpt.csv already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_Bpt.csv



In [43]:
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

#this_rds <- matched_PAs[18]
#id_pa <- basename(this_rds) %>% readr::parse_number() %>% unique()
#print(this_rds)

#matched <- readRDS(s3_get(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_pa_",id_pa,"_matching_results_wk",gediwk,".RDS",sep="")))
#nrow(matched)

#mras  <- tryCatch(matched2ras(matched),
#                      error=function(cond){
#                        message(cond)
#                        cat("Matched result is likely null for country", iso3,"pa", id_pa, "dimension of the match is", dim(matched),"\n")
#                        # writeLines("Matched results is likely null for country", paste(f.path,"WDPA_log/",iso3,"_log_matching.txt", sep=""))
#                        return(NULL)}) #convert the macthed df to a raster stack 
#print(table(mras$status[]))
#mras
#plot(mras)
#
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
#source(s3_get(paste(f.path,"vl_GEDI-PA_2024/matching_func_2024.R",sep="")))
#
#iso_matched_gedi <- extract_gedi(matched=matched, mras=mras, iso3=iso3)
#iso_matched_gedi <-  iso_matched_gedi %>%
#            dplyr::select("pa_id","status","shot_number", "lat_lowestmode", "lon_lowestmode", "wwfbiom","wwfecoreg",
#                          "pft","region", "rh25", "rh50", "rh75","rh90", "rh98", "agbd", "agbd_se")
#    if (length(unique(iso_matched_gedi$wwfbiom)) >1){
#        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% gsub("\\b(\\p{L})\\p{L}{2,}|.","\\U\\1",.,perl = TRUE)%>% str_c( collapse = "+")
#    } else if (length(unique(iso_matched_gedi$wwfbiom))==1){
#        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% gsub('\\b(\\p{L})\\p{L}{2,}|.','\\U\\1',.,perl = TRUE)
#    } else {
#        pabiome <- iso_matched_gedi$wwfbiom %>% unique()
#    }
#
#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
#
#dir.create(file.path(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,"/",sep="")))
#saveRDS(iso_matched_gedi, file=paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,"/",iso3,"_pa_", id_pa,"_gedi_wk_",gediwk,"_conti_", continent,"_biome_",pabiome,".RDS", sep=""))
#cat(id_pa,"in",iso3,"results is written to dir\n")


[1] "Bpt_pa_555576153_matching_results_wk24.RDS"
